In [71]:
import pandas as pd
import yaml
import numpy as np
import random
from math import log2, sqrt, exp
from dataset import Dataset

In [83]:
INPUT_PATH = 'data/wine_recognition/wine-recognition.tsv'
STRUCTURE_PATH = 'data/wine_recognition/metadata.yaml'
obj = Dataset(INPUT_PATH, STRUCTURE_PATH, '\t', 'target')
random.seed(1)
print(obj.data)

     target      1     2     3     4    5     6     7     8     9     10  \
0         1  14.23  1.71  2.43  15.6  127  2.80  3.06  0.28  2.29   5.64   
1         1  13.20  1.78  2.14  11.2  100  2.65  2.76  0.26  1.28   4.38   
2         1  13.16  2.36  2.67  18.6  101  2.80  3.24  0.30  2.81   5.68   
3         1  14.37  1.95  2.50  16.8  113  3.85  3.49  0.24  2.18   7.80   
4         1  13.24  2.59  2.87  21.0  118  2.80  2.69  0.39  1.82   4.32   
..      ...    ...   ...   ...   ...  ...   ...   ...   ...   ...    ...   
173       3  13.71  5.65  2.45  20.5   95  1.68  0.61  0.52  1.06   7.70   
174       3  13.40  3.91  2.48  23.0  102  1.80  0.75  0.43  1.41   7.30   
175       3  13.27  4.28  2.26  20.0  120  1.59  0.69  0.43  1.35  10.20   
176       3  13.17  2.59  2.37  20.0  120  1.65  0.68  0.53  1.46   9.30   
177       3  14.13  4.10  2.74  24.5   96  2.05  0.76  0.56  1.35   9.20   

       11    12    13  
0    1.04  3.92  1065  
1    1.05  3.40  1050  
2    1.03  3.17

In [84]:
def normalization(dataset,max_train,min_train): # Normalization of the dateset
        dataNorm=(dataset-min_train)/(max_train-min_train)
        return dataNorm

In [85]:
def Kfolds_normalization(data_kfolds,k):
    min_data_kfolds = []
    max_data_kfolds = []
    for i in range(k):
        idx = data_kfolds[i]['train'].columns
        if idx[0]=='target':
            mina = np.min(data_kfolds[i]['train'][idx[1:]])
            maxb = np.max(data_kfolds[i]['train'][idx[1:]])
            data_kfolds[i]['train'][idx[1:]] = normalization(data_kfolds[i]['train'][idx[1:]],mina,maxb)
            data_kfolds[i]['test'][idx[1:]]  = normalization(data_kfolds[i]['test'][idx[1:]],mina,maxb)
            min_data_kfolds.append(mina)
            max_data_kfolds.append(maxb)
        else:
            mina = np.min(data_kfolds[i]['train'][idx[0:-1]])
            maxb = np.max(data_kfolds[i]['train'][idx[0:-1]])
            data_kfolds[i]['train'][idx[0:-1]] = normalization(data_kfolds[i]['train'][idx[0:-1]],mina,maxb)
            data_kfolds[i]['test'][idx[0:-1]]  = normalization(data_kfolds[i]['test'][idx[0:-1]],mina,maxb)
            min_data_kfolds.append(mina)
            max_data_kfolds.append(maxb)
    return data_kfolds

In [87]:
k = 10 # k folds
data_kfolds = obj.get_n_class_k_final_folds(k)
data_kfolds_normalize = Kfolds_normalization(data_kfolds,k)
print(data_kfolds_normalize[0]['test'])

    target         1         2         3         4         5         6  \
50       1  0.468421  0.823529  0.721311  1.047619  0.760870  0.424908   
43       1  0.418421  0.350840  0.516393  0.744048  0.641304  0.454212   
39       1  0.160526  0.348739  0.336066  1.000000  0.369565  0.322344   
29       1  0.213158  0.834034  0.581967  0.833333  0.717391  0.450549   
56       1  0.160526  0.829832  0.508197  0.815476  0.478261  0.249084   
38       2  0.668421  0.890756  0.770492  0.833333  0.836957  0.487179   
28       2  0.836842  0.836134  0.245902  0.238095  0.804348  0.717949   
10       2  0.689474  0.936975  0.959016  0.785714  0.119565  0.743590   
45       2  0.610526  0.823529  0.770492  0.565476  0.836957  0.615385   
59       2  0.542105  0.466387  0.770492  0.833333  0.891304  0.824176   
66       2  0.726316  0.733193  0.614754  0.535714  0.836957  0.468864   
61       2  0.889474  0.682773  0.409836  0.595238  0.717391  0.358974   
25       3  0.436842  0.096639  0.4918

In [121]:
# setting of the parameters of neural networks
L = 4; # Nro of the layers in the neural networks
x = np.array([0,1,5,3,2]) # inputs
N_l= np.array([x.size,3,3,2],dtype=int) # array that content the number of the neurons for each layers
weights= {} # save the weights of the neural network
# we generate the weights randomly
for i in range(0,L-1):
    weights[i] = np.random.rand(N_l[i+1],N_l[i]+1)
    print(weights[i].shape)
alpha = 1

(3, 6)
(3, 4)
(2, 4)


In [119]:
def g(theta): # activation function
    return 1/(1+np.exp(-theta))

def Update_weight(weights,d_weights,aplha):
    for i in range(0,L-1):
        weights[i] = weights[i] - aplha*d_weights
    return weights

def forward(x,L,N_l,weights):
    for i in range(0,L-1):
        z = weights[i].dot(np.append(x,1))
        x = g(z)
    return x

In [122]:
print(forward(x,L,N_l,weights))

[0.84513554 0.91586646]


In [82]:
'''
INITIAL PARAMETERS

'''
INPUT_PATH = 'data/house_votes_84/house-votes-84.tsv'
STRUCTURE_PATH = 'data/house_votes_84/metadata.yaml'
obj = Dataset(INPUT_PATH, STRUCTURE_PATH, '\t', 'target')
random.seed(1)
k = 10 # k folds
data_kfolds = obj.get_n_class_k_final_folds(k) # we do k folds cross validation
data_kfolds_normalize = Kfolds_normalization(data_kfolds,k) # we normalize the k folds cross validation
print(data_kfolds_normalize[0]['test'])

     handicapped-infants  water-project-cost-sharing  \
159                  0.5                         0.5   
30                   0.5                         0.5   
109                 -0.0                         0.5   
70                  -0.0                         1.0   
22                   0.5                        -0.0   
103                 -0.0                         0.5   
136                  0.5                         0.5   
12                   0.5                        -0.0   
72                   0.5                        -0.0   
163                  0.5                         0.5   
130                  0.5                         0.5   
103                 -0.0                         0.5   
16                   0.5                         1.0   
23                   0.5                        -0.0   
160                 -0.0                        -0.0   
20                   0.5                        -0.0   
13                  -0.0                        